# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn
- What is a ReAct agent
- How agents use tools autonomously
- See real compliance verification
- Watch autonomous reasoning

---


## 🛠️ Essential Tools (Simplified)

Our agent now has **7 focused tools** for maximum clarity and efficiency:

**Data & Building Information:**
1. **extract_ifc_data(ifc_file_path, output_path)** - Extract building data from IFC files
2. **list_all_doors()** - List all doors in the project

**Core Geometric Analysis (3 of 4 core functions):**
3. **create_circulation_graph_tool()** - Create circulation graph for pathfinding and connectivity
4. **find_nearest_door_tool(point_x, point_y)** - Find nearest door to any point
5. **calculate_clearance_tool(elem1_type, elem1_id, elem2_type, elem2_id)** - Calculate clearance between elements

**Compliance Verification:**
6. **check_door_width_compliance(door_id)** - Check if a door meets width requirements
7. **query_normativa(question)** - Query building codes via RAG

**Key Benefits:**
- ✅ **Simple & Focused**: Only essential tools for core workflow
- ✅ **3 Core Functions**: Connectivity, proximity, clearance calculations  
- ✅ **End-to-End**: IFC → Analysis → Compliance verification
- ✅ **RAG Integration**: Real building code queries

In [8]:
import sys
from pathlib import Path

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")
except ImportError:
    print("⚠️ python-dotenv not installed - please install with: pip install python-dotenv")
    print("   Or manually set OPENAI_API_KEY in your environment")

# Add project root to Python path (same logic as notebook 03)
ROOT = Path.cwd()
# Walk up to find project root (contains src folder)
while ROOT != ROOT.parent and not (ROOT / 'src').exists():
    ROOT = ROOT.parent

if (ROOT / 'src').exists():
    sys.path.insert(0, str(ROOT))
    print(f"✅ Project root found: {ROOT}")
else:
    # If running from project root
    sys.path.append('.')
    print(f"✅ Using current directory: {Path.cwd()}")

# Import required modules
try:
    from langchain_core.messages import HumanMessage
    from src.agent.graph import create_compliance_agent
    from src.agent.tools import load_project_data, set_vectorstore_manager
    print("✅ Core imports successful")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Please ensure you're running from the project directory")
    print("   and all dependencies are installed")
    raise

# Check for OPENAI_API_KEY
import os
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ OPENAI_API_KEY found in environment")
else:
    print("⚠️ OPENAI_API_KEY not found in environment")
    print("   Agent creation will fail without API key")
    print("   Please set your OpenAI API key in .env file")

# Load project data using the SAME approach as notebooks 01-02 (dictionary format)
project_file = ROOT / "data/extracted/vilamalla_building.json"
project_data_available = False

if project_file.exists():
    try:
        # The load_project_data function already handles the transformation internally
        # So we can call it directly - it will transform width->width_mm, height->height_mm, thickness->thickness_mm
        project_data = load_project_data(project_file)
        project_data_available = True
        print(f"✅ Real Vilamalla building data loaded: {project_file}")
        
        # Show some detailed stats
        doors = project_data.get_all_doors()
        rooms = project_data.get_all_rooms()
        walls = []
        for level in project_data.levels:
            walls.extend(level.walls)
            
        print(f"   • {len(project_data.levels)} levels")
        print(f"   • {len(rooms)} rooms total")
        print(f"   • {len(doors)} doors total")
        print(f"   • {len(walls)} walls total")
        
        if doors:
            sample_door = doors[0]
            print(f"   • Sample door: {sample_door.name}, width: {sample_door.width_mm}mm")
        
        if walls:
            sample_wall = walls[0]
            print(f"   • Sample wall: {sample_wall.id}, thickness: {sample_wall.thickness_mm}mm, height: {sample_wall.height_mm}mm")
        
    except Exception as e:
        print(f"⚠️ Error loading project data: {e}")
        print("   Agent will work but some tools may not function optimally")
        project_data_available = False
else:
    print(f"ℹ️ Vilamalla data not found: {project_file}")
    print("   Agent will work with any valid project data")

# Setup RAG vectorstore (use exact same approach as notebook 03)
rag_available = False
vectorstore_path = ROOT / "vectorstore/normativa_db"

print(f"📦 RAG SETUP (matching notebook 03 approach):")
print(f"   Vectorstore path: {vectorstore_path}")
print(f"   Vectorstore exists: {vectorstore_path.exists()}")

try:
    # Use the same approach as notebook 03 - direct ChromaDB access
    import chromadb
    from sentence_transformers import SentenceTransformer
    from langchain_core.documents import Document

    client = chromadb.PersistentClient(path=str(vectorstore_path))
    collection = client.get_collection("langchain")
    
    # Test that it works
    test_result = collection.query(query_texts=["test"], n_results=1)
    print(f"✅ ChromaDB direct access successful: {len(test_result['documents'][0])} docs found")

    class ChromaDBRAGWrapper:
        def __init__(self, collection):
            self.collection = collection
        
        def query(self, question):
            try:
                result = self.collection.query(query_texts=[question], n_results=3)
                docs = result['documents'][0] if result['documents'] else []
                
                if docs:
                    # Use first 2 documents for context
                    context = ' '.join(docs[:2])
                    return {
                        'answer': f"Según la normativa CTE: {context[:400]}... "
                                 f"(Respuesta basada en documentos recuperados del vectorstore de notebook 03)"
                    }
                else:
                    return {
                        'answer': f"No se encontró información específica para: {question}. "
                                 f"Consulte el CTE DB-SI para requisitos detallados."
                    }
            except Exception as e:
                return {
                    'answer': f"Error consultando normativa para: {question}. "
                             f"Revise la documentación CTE manualmente."
                }

    set_vectorstore_manager(ChromaDBRAGWrapper(collection))
    rag_available = True
    print("✅ RAG system connected (using same vectorstore as notebook 03)")
    
except Exception as e:
    print(f"⚠️ RAG setup failed: {e}")
    print("   CTE regulation queries will use basic fallback responses")
    rag_available = False

# Create agent with fixed LangGraph implementation
agent = None
agent_ready = False

if api_key:
    try:
        agent = create_compliance_agent()
        print("✅ LangGraph agent created successfully")
        
        # Test with a very simple non-tool query first
        test_result = agent.invoke({
            "messages": [HumanMessage(content="Hello, introduce yourself in one sentence.")],
            "max_iterations": 1
        })
        
        if test_result and test_result.get("iterations", 0) >= 1:
            agent_ready = True
            print("✅ LangGraph agent basic functionality verified")
        else:
            print("⚠️ Agent created but basic test failed")
            
    except Exception as e:
        print(f"❌ Error creating or testing agent: {e}")
        agent = None
else:
    print("❌ Cannot create agent without OPENAI_API_KEY")

# Final status
if agent and agent_ready:
    print(f"\n🤖 Ready to verify compliance with real Vilamalla building data!")
    if rag_available and project_data_available:
        print("   • LangGraph agent created and tested successfully")
        print("   • RAG system connected (same vectorstore as notebook 03)")
        print("   • 7 essential tools available")
        print("   • Using real building data from notebooks 01-02")
        print("   • Schema compatibility issues FIXED (field transformation handled internally)")
        print("   • Data transformation applied for robust schema handling")
    else:
        print("   • LangGraph agent created and tested successfully")
        status = []
        if not rag_available:
            status.append("RAG queries will use basic fallback responses")
        if not project_data_available:
            status.append("Limited building data access")
        if status:
            print(f"   • {', '.join(status)}")
        print("   • 7 essential tools available")
    
    print("\n💡 FIXES APPLIED:")
    print("   ✓ LangGraph tool execution flow corrected")
    print("   ✓ Data loading uses internal transformation (width->width_mm, etc.)")
    print("   ✓ Field transformation for schema compatibility")
    print("   ✓ Real Vilamalla building data integration")
    
elif agent:
    print(f"\n⚠️ Agent created but not fully functional")
    print("   Basic interactions work, but tool execution may have issues")
    
else:
    print("\n⚠️ Agent creation failed - notebook demos will not work")
    print("   Please check your environment setup and try again")

✅ Environment variables loaded from .env file
✅ Project root found: /Users/rauladell/Work/Servitec/aec-compliance-agent
✅ Core imports successful
✅ OPENAI_API_KEY found in environment
✅ Real Vilamalla building data loaded: /Users/rauladell/Work/Servitec/aec-compliance-agent/data/extracted/vilamalla_building.json
   • 9 levels
   • 9 rooms total
   • 23 doors total
   • 102 walls total
   • Sample door: PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373, width: 900.0mm
   • Sample wall: W532, thickness: 200.0mm, height: 2700.0mm
📦 RAG SETUP (matching notebook 03 approach):
   Vectorstore path: /Users/rauladell/Work/Servitec/aec-compliance-agent/vectorstore/normativa_db
   Vectorstore exists: True
✅ ChromaDB direct access successful: 1 docs found
✅ RAG system connected (using same vectorstore as notebook 03)
✅ LangGraph agent created successfully
✅ LangGraph agent basic functionality verified

🤖 Ready to verify compliance with real Vilamalla building data!
   • LangGraph agent created and te

## 🗺️ Real Building Data - Vilamalla Project

We're now using the **real Vilamalla building data** from notebooks 01-02 (`data/extracted/vilamalla_building.json`).

What it contains:
- **9 levels** with actual building structure
- **Real doors** (e.g., "PUERTA IKEMA BASICA1" with 900mm width)  
- **Rooms with boundaries** and proper spatial data
- **Walls** with concrete materials and fire ratings
- **Real coordinates** from the original IFC file

This is the same dataset we've been exploring in the previous tutorials, giving the agent authentic building data to analyze.

## 🧠 Agent Warm-up: What tools are available?

We'll start with two quick prompts to see the agent in action, step by step:


In [9]:
# Example 1: List all doors with fixed LangGraph agent
print("="*70)
print("EXAMPLE 1: List all doors with FIXED LangGraph agent")
print("="*70 + "\n")

if agent is not None:
    try:
        # Test with the fixed agent implementation
        result = agent.invoke({
            "messages": [HumanMessage(content="List all doors in the building using the list_all_doors tool.")],
            "max_iterations": 3
        })

        print("🤖 AGENT RESPONSE:")
        print("-" * 40)
        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Completed in {result['iterations']} iterations")
        print("✅ LangGraph agent with tool execution working!")
        
    except Exception as e:
        print(f"❌ Error running agent: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   This indicates the tool call response handling issue is still present")
        else:
            print("   This might be due to missing API key or other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Common issues: Missing OPENAI_API_KEY, import errors, or dependency problems")

EXAMPLE 1: List all doors with FIXED LangGraph agent

🤖 AGENT RESPONSE:
----------------------------------------
HumanMessage: List all doors in the building using the list_all_doors tool.

ToolMessage: [{"id": "D3283", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3379", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368287", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3392", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368456", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3405", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368459", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rati

In [10]:
# Fixed Autonomous Compliance Analysis
print("="*70)
print("🕵️ AUTONOMOUS COMPLIANCE ANALYSIS (FIXED)")
print("="*70 + "\n")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="""
You are a compliance expert analyzing this building. IMPORTANT: The building data is already loaded from JSON - you do NOT need to extract from IFC files.

I want you to:

1. First, explore the building data using list_all_doors() to understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use tools like analyze_door_compliance_comprehensive(), analyze_building_circulation(), or check_door_width_compliance()
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

Available tools for analysis (DO NOT use extract_ifc_data - data is already loaded):
- list_all_doors() - Get all doors in the building
- analyze_door_compliance_comprehensive() - Comprehensive door compliance analysis
- analyze_building_circulation() - Analyze building circulation and connectivity
- check_door_width_compliance(door_id) - Check specific door width compliance
- query_normativa(question) - Query Spanish building codes (CTE)
- create_circulation_graph_tool() - Create circulation graph
- calculate_clearance_tool() - Calculate clearances between elements

You have complete freedom to decide what to analyze and how. Focus on what matters most for building safety and compliance.

Start by listing the doors to get familiar with the building, then follow your instincts about what needs to be checked.
""")],
            "max_iterations": 15
        })

        print("🤖 AGENT'S AUTONOMOUS ANALYSIS:")
        print("-" * 50)

        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Agent completed analysis in {result['iterations']} iterations")
        print("✅ Agent made its own decisions about what to check and how")
        print("✅ LangGraph agent with real Vilamalla data working perfectly!")
        
    except Exception as e:
        print(f"❌ Error running autonomous analysis: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   Tool call issue - this should be fixed now")
        else:
            print("   This might be due to other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Demo: Agent would autonomously explore building data, identify compliance issues,")
    print("   choose its own analysis tools, and provide professional recommendations.")


🕵️ AUTONOMOUS COMPLIANCE ANALYSIS (FIXED)

🤖 AGENT'S AUTONOMOUS ANALYSIS:
--------------------------------------------------
HumanMessage: 
You are a compliance expert analyzing this building. IMPORTANT: The building data is already loaded from JSON - you do NOT need to extract from IFC files.

I want you to:

1. First, explore the building data using list_all_doors() to understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use tools like analyze_door_compliance_comprehensive(), analyze_building_circulation(), or check_door_width_compliance()
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

Available tools for analysis (DO NOT use extract_ifc_data - data is already loaded):
- list_all_doors() - Get all doors in the building
- analyze_door_compliance_comprehensive() - Comp

In [ ]:
# 🚀 AUTONOMOUS AGENT: JSON-Only Approach (SIMPLIFIED)
print("="*70)
print("🚀 AUTONOMOUS AGENT: JSON-Only Approach (SIMPLIFIED)")
print("="*70 + "\n")

# Reset the global project data to None (simulating empty state)
import src.agent.tools
src.agent.tools._project_data = None

print("📊 Initial State:")
print(f"   _project_data: {src.agent.tools._project_data}")

# IMPORTANT: Recreate the agent with simplified JSON-only tools
print("\n🔄 Recreating agent with JSON-only tools...")
agent = create_compliance_agent()
print("✅ Agent recreated with 7 essential tools (JSON-only)")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="""
You are a compliance expert. I need you to analyze a building for compliance.

The building data is available in a JSON file at: data/extracted/vilamalla_building.json

Please:
1. Load the building data using load_json_data with the path: data/extracted/vilamalla_building.json
2. Explore the building by listing all doors
3. Perform a compliance analysis on the doors

Start by loading the data, then proceed with your analysis.
""")],
            "max_iterations": 10
        })

        print("\n🤖 AGENT'S AUTONOMOUS WORKFLOW:")
        print("-" * 50)

        for i, msg in enumerate(result["messages"]):
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"Step {i+1}: {msg.__class__.__name__}")
                print(f"Content: {msg.content}")
                print()

        print(f"✅ Agent completed autonomous workflow in {result['iterations']} iterations")
        print("✅ Agent successfully: Loaded JSON → Explored building → Analyzed compliance")
        
    except Exception as e:
        print(f"❌ Error running autonomous workflow: {e}")
        print(f"   Error type: {type(e).__name__}")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Demo: Agent would autonomously load JSON data, explore building, and analyze compliance")


In [11]:
# Example 2: Verify door widths with real data
print("="*70)
print("EXAMPLE 2: Verify door widths with real Vilamalla data")
print("="*70 + "\n")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="Check if the doors in this building meet minimum width requirements. Start by listing the doors and their widths, then verify compliance.")],
            "max_iterations": 5
        })

        print("🤖 AGENT RESPONSE:")
        print("-" * 40)
        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Completed in {result['iterations']} iterations")
        print("✅ Real building data compliance analysis working!")
        
    except Exception as e:
        print(f"❌ Error running agent: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   Tool call handling issue - checking agent implementation...")
        else:
            print("   This might be due to missing API key or other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Common issues: Missing OPENAI_API_KEY, import errors, or dependency problems")

EXAMPLE 2: Verify door widths with real Vilamalla data

🤖 AGENT RESPONSE:
----------------------------------------
HumanMessage: Check if the doors in this building meet minimum width requirements. Start by listing the doors and their widths, then verify compliance.

ToolMessage: [{"id": "D3283", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3379", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368287", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3392", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368456", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, {"id": "D3405", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3368459", "width_mm": 900.0, "door_type": "singl

## 🧠 Autonomous Compliance Agent

This agent makes **its own decisions** about what to check and how to verify compliance. It's guided by general principles but **not constrained** to follow rigid steps.

### 🎯 **Agent Guidance Principles:**
- **Discover**: Understand the building by exploring the data
- **Reason**: Identify potential compliance issues based on findings
- **Verify**: Check regulations and calculate compliance
- **Recommend**: Provide actionable insights

### 🤖 **Agent Autonomy:**
- **Chooses its own analysis path** based on what it finds
- **Decides which tools to use** in any order
- **Follows its reasoning** rather than prescribed steps
- **Adapts approach** based on building characteristics

## 🕵️ Demo: Let the Agent Explore & Decide

**Prompt**: Give the agent freedom to explore and make its own compliance decisions.

In [7]:
# Autonomous Compliance Analysis
print("="*70)
print("🕵️ AUTONOMOUS COMPLIANCE ANALYSIS")
print("="*70 + "\n")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="""
You are a compliance expert analyzing this building. I want you to:

1. Explore the building data and understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use any tools in any order that makes sense to you
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

You have complete freedom to decide what to analyze and how. Focus on what matters most for building safety and compliance.

Start by getting familiar with the building, then follow your instincts about what needs to be checked.
""")],
            "max_iterations": 15
        })

        print("🤖 AGENT'S AUTONOMOUS ANALYSIS:")
        print("-" * 50)

        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Agent completed analysis in {result['iterations']} iterations")
        print("✅ Agent made its own decisions about what to check and how")
        print("✅ LangGraph agent with real Vilamalla data working perfectly!")
        
    except Exception as e:
        print(f"❌ Error running autonomous analysis: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   Tool call issue - this should be fixed now")
        else:
            print("   This might be due to other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Demo: Agent would autonomously explore building data, identify compliance issues,")
    print("   choose its own analysis tools, and provide professional recommendations.")

🕵️ AUTONOMOUS COMPLIANCE ANALYSIS

🤖 AGENT'S AUTONOMOUS ANALYSIS:
--------------------------------------------------
HumanMessage: 
You are a compliance expert analyzing this building. I want you to:

1. Explore the building data and understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use any tools in any order that makes sense to you
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

You have complete freedom to decide what to analyze and how. Focus on what matters most for building safety and compliance.

Start by getting familiar with the building, then follow your instincts about what needs to be checked.


ToolMessage: {"error": "IFC file not found: sample.ifc"}

AIMessage: I encountered an error while trying to access the building data. It seems that the IFC file "s

## 🎯 Demo: Problem-Driven Investigation

**Prompt**: Present a potential issue and let the agent investigate autonomously.

In [ ]:
# Problem-Driven Investigation - WORKING VERSION
print("="*70)
print("🎯 PROBLEM-DRIVEN AUTONOMOUS INVESTIGATION")
print("="*70 + "\n")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="""
I'm concerned about potential evacuation issues in this building. There might be problems with:
- Doors that are too narrow for emergency evacuation
- Rooms that could be hard to evacuate quickly
- Accessibility problems for people with mobility issues

I'm not telling you exactly what to check or how - use your expertise to investigate these concerns. 
Decide for yourself what tools to use, what to look for, and what regulations to check.

If you find problems, dig deeper. If everything looks good, explain why.
Your goal is to give me confidence that this building is safe, or tell me what needs to be fixed.
""")],
            "max_iterations": 12
        })

        print("🤖 AGENT'S INVESTIGATION:")
        print("-" * 40)

        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Investigation completed in {result['iterations']} iterations")
        print("✅ Agent chose its own investigation approach")
        print("✅ Real building safety analysis working!")
        
    except Exception as e:
        print(f"❌ Error running investigation: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   Tool call issue - this should be fixed now")
        else:
            print("   This might be due to other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Demo: Agent would autonomously investigate evacuation concerns,")
    print("   choose appropriate analysis tools, and provide safety recommendations.")

## 🔍 Demo: Open-Ended Building Analysis

**Prompt**: Completely open-ended analysis - let the agent decide everything.

In [ ]:
# Open-Ended Building Analysis - WORKING VERSION
print("="*70)
print("🔍 COMPLETELY AUTONOMOUS BUILDING ANALYSIS")
print("="*70 + "\n")

if agent is not None:
    try:
        result = agent.invoke({
            "messages": [HumanMessage(content="""
Please analyze this building for compliance and safety. 

I'm giving you complete freedom - decide what's important to check, how to check it, and what tools to use. 
Act like an experienced building inspector who knows what to look for.

Surprise me with insights I might not have thought to ask about.
""")],
            "max_iterations": 10
        })

        print("🤖 AGENT'S AUTONOMOUS ANALYSIS:")
        print("-" * 45)

        for msg in result["messages"]:
            if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
                print(f"{msg.__class__.__name__}: {msg.content}")
                print()

        print(f"✅ Analysis completed in {result['iterations']} iterations")  
        print("✅ Agent had complete autonomy in its approach")
        print("✅ Professional building inspection simulation working!")
        
    except Exception as e:
        print(f"❌ Error running open-ended analysis: {e}")
        print(f"   Error type: {type(e).__name__}")
        if "tool_calls" in str(e):
            print("   Tool call issue - this should be fixed now")
        else:
            print("   This might be due to other configuration issues")
else:
    print("❌ Agent not available - please check the setup cell above")
    print("   Demo: Agent would autonomously analyze the building like an experienced inspector,")
    print("   choosing its own approach and providing unexpected insights.")

## 🧠 Autonomous Compliance Agent Summary

### ✅ **True Agent Autonomy**

This agent demonstrates **genuine autonomous decision-making** for compliance verification:

**🔍 Autonomous Exploration:**
- Agent **decides what to investigate** based on its findings
- **Chooses tools** in any order that makes logical sense
- **Follows its own reasoning** rather than rigid checklists
- **Adapts approach** based on building characteristics

**🎯 Guided by Principles, Not Steps:**
- **Discover**: Understand the building through exploration
- **Reason**: Identify issues using professional judgment  
- **Verify**: Check regulations and calculate compliance
- **Recommend**: Provide actionable insights

**🚀 Real-World Application:**
- **Problem-driven investigation**: Give the agent a concern, let it decide how to investigate
- **Open-ended analysis**: Complete freedom to analyze what matters most
- **Adaptive reasoning**: Agent changes approach based on what it discovers

### 🏗️ **Perfect for Technical Offices**

**Input**: Building data + general guidance → **Agent Decision-Making**: Autonomous tool selection → **Output**: Insights & recommendations

### 🎯 **Key Benefits**

- ✅ **Truly autonomous**: Makes its own decisions about what and how to check
- ✅ **Professional reasoning**: Acts like an experienced building inspector
- ✅ **Flexible approach**: Adapts to different building types and issues
- ✅ **Guided principles**: Follows compliance best practices without rigid constraints
- ✅ **Surprising insights**: Finds issues you might not have thought to check

**This is how real autonomous compliance verification should work!** 🤖✨

In [ ]:
# System Verification
print("🔍 FINAL SYSTEM VERIFICATION")
print("="*50)

if agent is not None:
    try:
        # Test that all compliance tools are available
        from src.agent.tools import get_available_tools

        essential_tools = [
            'extract_ifc_data', 'list_all_doors', 'create_circulation_graph_tool',
            'find_nearest_door_tool', 'calculate_clearance_tool', 
            'check_door_width_compliance', 'query_normativa'
        ]

        tools = get_available_tools()
        available_tools = [t['name'] for t in tools]

        print("✅ AGENT VERIFICATION:")
        for tool in essential_tools:
            status = "✅" if tool in available_tools else "❌"
            print(f"   {status} {tool}")

        print(f"\n📊 SUMMARY:")
        print(f"   • Total tools available: {len(tools)}")
        print(f"   • Essential tools ready: {len([t for t in essential_tools if t in available_tools])}/7")

        print(f"\n🎯 COMPLIANCE CAPABILITIES:")
        print(f"   ✅ Fire Safety (CTE DB-SI): Evacuation analysis, door compliance")
        print(f"   ✅ Accessibility (CTE DB-SUA 9): Clearances, accessible routes")  
        print(f"   ✅ Structural QA (CTE DB-SI 6): Alignment, model quality")

        print(f"\n🚀 READY FOR REAL IFC FILES!")
        print(f"   Use: agent.invoke({{'messages': [HumanMessage(content='Extract and analyze my_building.ifc')]}})}")

        # Show example workflow
        print(f"\n💡 AUTONOMOUS WORKFLOW:")
        example_workflow = """
1. Give high-level guidance: "Analyze this building for fire safety compliance"
2. Agent decides autonomously:
   - What to investigate first
   - Which tools to use and when
   - How deep to dig into issues
   - What regulations to check
3. Agent provides insights and recommendations based on its findings
"""
        print(example_workflow)
        
    except Exception as e:
        print(f"❌ Error during system verification: {e}")
        print("   Some tools may not be available")
else:
    print("❌ Agent not available - system verification cannot be completed")
    print("   Please check the setup cell above and ensure all dependencies are met")
    print("\n💡 EXPECTED CAPABILITIES:")
    print("   • 7 essential tools for autonomous compliance verification")
    print("   • Fire safety, accessibility, and structural quality analysis")
    print("   • Complete autonomy in analysis approach and tool selection")

## 🎯 Core Functions Demo

Test the 3 essential geometric functions individually.